Este script realiza seguimiento de la información WhoIs y MX de los dominios que se listen en un google sheets.
Para comenzar a utilizarlo:<br>
1) Editar la URL del Google sheets que se use (primera columna, a partir de la segunda fila pegar los dominios)<br>
2) Editar en el código el número máximo de filas a considerar (muy importante!)<br><br>

-> Pueden haber límites de consultas diarias a las APIs.

In [22]:
!pip3 install python-whois
!pip3 install dnspython
!pip3 install gspread-pandas
!pip3 install gspread-dataframe
!pip3 install gspread_formatting

In [23]:
import whois
import dns
import dns.resolver
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import matplotlib.pyplot as plt
import time
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting.dataframe import format_with_dataframe

In [33]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
# CAMBIAR POR URL DEL GOOGLE SHEETS QUE SE ESTÉ UTILIZANDO
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1GDcZVTmdlx485bVz0jbB1OTNhCYZPsyRS_eE8FBBOwo/edit#gid=0')
# Se toma la hoja que se llama "dominios"
sheet = wb.worksheet('dominios')
data = sheet.get_all_values()

df = pd.DataFrame(data)
df.columns = df.iloc[0]

# CAMBIAR SEGÚN EL NÚMERO DE FILAS QUE SE QUIERA INCLUIR EJ: [1:4], TOMARÁ DE FILAS 1 A LA 4
df = df.iloc[1:926]

In [56]:
for i in range(len(df)):
  domain_name = df.iloc[i][0]
  domain='www.'+ domain_name
  domain_info = whois.whois(domain)
 # print(domain_info)
  # sheet.update_cell(i+2,2, domain_info.get('registrant_name', 'N/A'))
  try:
    df.at[i+1,'Titular']=domain_info.get('registrant_name', 'N/A')
    # Get name servers
    ns1, ns2 = domain_info.get('name_servers', 'N/A')[0], domain_info.get('name_servers', 'N/A')[1]
    # sheet.update_cell(i+2,3, ns1)
    df.at[i+1,'DNS1']=ns1
    # sheet.update_cell(i+2,4, ns2)
    df.at[i+1,'DNS2']=ns2
  except:
    print("Error")
  try:
    answers = dns.resolver.query(domain_name, 'MX')
    #index (desde qué n° de columna se empiezan a pegar los MX)
    j=4
    for rdata in answers:
      # sheet.update_cell(i+2,j, rdata.exchange.to_text())
      try:
        df.iat[i,j]=rdata.exchange.to_text()
      except:
        print("Error")
      # sheet.update_cell(i+2,j+1, rdata.preference)
      try:
        df.iat[i,j+1]=rdata.preference
        j=j+2
      except:
        print("Error")
  except:    
    print("Error detectado")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: please use dns.resolver.resolve() instead


Error detectado
Error
Error
Error
Error
Error detectado
Error detectado
Error
Error
Error
Error
Error detectado
Error detectado
Error
Error detectado
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error detectado
Error detectado
Error detectado


In [57]:
set_with_dataframe(sheet, df)
format_with_dataframe(sheet, df, include_column_header=True)

{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}],
 'spreadsheetId': '1GDcZVTmdlx485bVz0jbB1OTNhCYZPsyRS_eE8FBBOwo'}